# Criação do Dataset de Treino (Point-in-Time Correct)
Neste notebook, montamos o dataset final para treinamento do modelo.
Utilizamos o **Feast** para realizar o "Time-Travel Join", garantindo que as features associadas a cada transação reflitam o estado dos dados *no momento exato* em que a transação ocorreu, evitando vazamento de dados (data leakage).

In [1]:
import polars as pl
from feast import FeatureStore
from pathlib import Path
from tqdm import tqdm

# Configuração
REPO_PATH = "./feature_repo"
DATA_PATH = Path("./data")
TRAIN_DATA_PATH = DATA_PATH / "training_dataset.parquet"

store = FeatureStore(repo_path=REPO_PATH)

## 1. Carregamento das Entidades (Transações)
Carregamos as transações históricas que servirão como nossa tabela de entidades.
É fundamental ter a coluna `event_timestamp` para que o Feast saiba "quando" buscar as features.

In [ ]:
print("🚀 1. Carregando histórico de transações (Positivos)...")

# Carrega dataset de transações
entity_df = pl.read_parquet(DATA_PATH / "transactions_train.parquet", columns=['t_dat', 'customer_id', 'article_id'])
entity_df = entity_df.with_columns(
    pl.col("t_dat").cast(pl.Datetime).alias("event_timestamp")
)
# Usamos tail() para pegar as transações mais recentes, que alinham com as features
entity_df = entity_df.drop("t_dat").tail(300000)

print(f"📦 Entidades base carregadas: {entity_df.shape}")

🚀 1. Carregando histórico de transações (Positivos)...
📦 Entidades base carregadas: (300000, 3)


## 2. Enriquecimento com Features (Time-Travel Join)
Aqui acontece a mágica do Feature Store. Solicitamos ao Feast que anexe as features de usuário e item a cada transação.
O Feast busca o valor da feature válido no `event_timestamp` da transação.

In [ ]:
print("⏳ 2. Invocando Feast para Time-Travel Join...")

# Cria dataset de treinamento ao obter as user features
# avg_spend e purchase_count, e as item features
# popularity_score e avg_price
training_df = store.get_historical_features(
    entity_df=entity_df.to_pandas(),
    features=[
        "user_stats:avg_spend",
        "user_stats:purchase_count",
        "item_stats:popularity_score",
        "item_stats:avg_price"
    ]
).to_df()

training_df.head()

⏳ 2. Invocando Feast para Time-Travel Join...


,customer_id,article_id,event_timestamp,avg_spend,purchase_count,popularity_score,avg_price
0,f882d43eb7aab25667917f0ddbc18e994bd31538aa10e2...,0825509004,2020-09-13 00:00:00+00:00,0.029065,49,42,0.012804
1,ec6d1328d7b8091dddd9b769323c2f55c4f560053d583c...,0901575001,2020-09-13 00:00:00+00:00,0.022451,77,20,0.029129
2,f1e428069ea3a08ad75c607edd556552c6fc7c3609d121...,0695325020,2020-09-13 00:00:00+00:00,0.024136,7,192,0.023410
3,f04f22807493a3a0d57dec80326396765ed5b3f80f7f7e...,0872600009,2020-09-13 00:00:00+00:00,0.021280,23,202,0.026835
4,f18cef960f8901b371958dd413e372ee21d221574d006a...,0870957002,2020-09-13 00:00:00+00:00,0.031993,76,40,0.024872


## 3. Tratamento de Valores Nulos
Features podem vir nulas se, no momento da transação, o usuário ou item não tinha histórico (Cold Start). Preenchemos com 0.

In [ ]:
print("🧹 3. Limpeza e Preenchimento de Nulos...")

# Feature Stores geram Nulos se o usuário era novo na época.
training_df.fillna(0, inplace=True)

🧹 3. Limpeza e Preenchimento de Nulos...


## 4. Pré-processamento Final e Salvamento
Modelos de Deep Learning (como Two-Tower) precisam de IDs numéricos contínuos para suas camadas de Embedding.
Realizamos o Label Encoding dos IDs e salvamos os metadados (total de usuários/itens) para configurar a arquitetura da rede neural posteriormente.

In [5]:
# Encode de IDs para Inteiros (Necessário para Embeddings do PyTorch)
# Em produção, salvaríamos esses encoders como artefatos (Pickle/JSON)
print("🔢 4. Codificando IDs (Label Encoding)...")
training_df['user_index'] = training_df['customer_id'].astype('category').cat.codes
training_df['item_index'] = training_df['article_id'].astype('category').cat.codes

print(f"💾 Salvando Dataset de Treino pronto: {TRAIN_DATA_PATH}")
training_df.to_parquet(TRAIN_DATA_PATH)

# Salvar metadados para o modelo saber o tamanho dos embeddings
meta = {
    'num_users': int(training_df['user_index'].max() + 1),
    'num_items': int(training_df['item_index'].max() + 1)
}
import json
with open(DATA_PATH / "model_metadata.json", "w") as f:
    json.dump(meta, f)

print("✅ Concluído! Metadados:", meta)

🔢 4. Codificando IDs (Label Encoding)...
💾 Salvando Dataset de Treino pronto: data/training_dataset.parquet
✅ Concluído! Metadados: {'num_users': 14761, 'num_items': 8451}
